In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms,datasets
import torchvision.models as models
import numpy as np
from math import floor
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt

In [20]:
class SubLoader(torchvision.datasets.CIFAR10):
    def __init__(self, *args, exclude_list=[], **kwargs):
        super(SubLoader, self).__init__(*args, **kwargs)

        if exclude_list == []:
            return

        if self.train:
            labels = np.array(self.targets)
            exclude = np.array(exclude_list).reshape(1, -1)
            mask = ~(labels.reshape(-1, 1) == exclude).any(axis=1)

            self.data = self.data[mask]
            self.targets = labels[mask].tolist()
        else:
            labels = np.array(self.targets)
            exclude = np.array(exclude_list).reshape(1, -1)
            mask = ~(labels.reshape(-1, 1) == exclude).any(axis=1)

            self.data = self.data[mask]
            self.targets = labels[mask].tolist()

In [34]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

# testset = SubLoader(exclude_list=[i for i in range(2,10)],root='./data', train=False,download=True, transform=transform)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=100,shuffle=False, num_workers=2)

Files already downloaded and verified


In [5]:
# FGSM attack code
def fgsm_attack(img, eps, data_grad):
    perturbed_img = img + eps*data_grad.sign() # Perturbing image by epsilon shift along element-wise sign of the data gradient
    perturbed_img = torch.clamp(perturbed_img, 0, 1) # As MNIST data must be in [0,1]

    return perturbed_img

In [40]:
alexnet = torch.hub.load('pytorch/vision', 'alexnet', pretrained=True)
alexnet.eval()

AttributeError: module 'torch' has no attribute 'hub'

In [26]:
def test( model, test_loader, eps ):

    # Accuracy counter
    correct = 0
    adv_examples = []

    for data, label in test_loader:
        data.requires_grad = True # Set flag to enable computation of data_grad
        output = model(data)  # Forward pass 
        _, init_pred = torch.max(outputs.data, 1) # get the initial prediction

        if init_pred.item() != label.item():  # Only bother to attck if initial prediction is correct
            continue
        
        criterion = nn.CrossEntropyLoss() #defining loss function 
        loss = criterion(output, label) # Calculate the cross entropy loss
        
        model.zero_grad() # Zero all existing gradients (prevents accumulation of gradient like in RNNs)
        loss.backward() # Calculate gradients in backward pass
       
        data_grad = data.grad.data  # data_grad
        perturbed_data = fgsm_attack(data, eps, data_grad) # FGSM Attack
        
        output = model(perturbed_data) # Re-classify the perturbed image

        _,final_pred = torch.max(outputs.data, 1) # get the new prediction
        
        if final_pred.item() == label.item(): # Check for success
            correct += 1
            
            if (eps == 0) and (len(adv_examples) < 5):
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:    
            # Saving adversarial examples
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )

    # Calculate final accuracy for this epsilon
    final_acc = correct/float(len(test_loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(eps, correct, len(test_loader), final_acc))

    # Return the accuracy and an adversarial examples
    return final_acc, adv_examples


In [27]:
epsilons = [0, .05, .1, .15, .2, .25, .3]
model = alexnet
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    acc, ex = test(model, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

RuntimeError: invalid argument 2: size '[4 x 9216]' is invalid for input with 50176 elements at /pytorch/aten/src/TH/THStorage.c:41

In [ ]:
# Plot several examples of adversarial samples at each epsilon
cnt = 0
plt.figure(figsize=(8,10))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig,adv,ex = examples[i][j]
        plt.title("{} -> {}".format(orig, adv))
        plt.imshow(ex, cmap="gray")
plt.tight_layout()
plt.show()

In [4]:
#Define AlexNet
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 2 * 2)
        x = self.classifier(x)
        return x